In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
import genetic_alg as ga

In [3]:
from dataset import load_dataset, load_labels, convert_to_epochs, load_channels
from features import time_series_features, hjorth_features
from classifiers import KNN, SVM, NN
import variables as v

In [4]:
#importing ICA filtered two times

dataset_ica_2_ = load_dataset(data_type="ica2", test_type="Arithmetic")

channels = load_channels()

print(len(channels))


32


In [5]:
channels_0  = ['F3', 'C3', 'P3', 'O1', 'O2', 'P4', 'C4', 'F4']
channels_1  = ['Fp1', 'C3', 'P7', 'O1', 'O2', 'P8', 'C4', 'Fp2']
channels_2  = ['Cz', 'Fz', 'P3', 'Pz', 'O1', 'Oz', 'O2', 'P4']
channels_3  = ['Cz', 'Fz', 'T7', 'P3', 'O1', 'O2', 'P4', 'T8']
channels_4  = ['F7', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'F8']
channels_5  = ['FC1', 'FC5', 'FT9', 'CP5', 'CP1', 'PO9', 'T8', 'C4']
channels_6  = ['F3', 'P7', 'PO10', 'CP2', 'CP6', 'FT10', 'FC6', 'FC2']
channels_7  = ['Fz', 'F7', 'FC1', 'T7', 'Pz', 'P8','FC6','Fp2']
channels_8  = ['FT9', 'CP1', 'PO9', 'Pz', 'CP2', 'P8', 'FT10', 'FP2']
channels_9  = ['Cz', 'C3', 'FC5', 'P3', 'P7','O2', 'T8','FC2']
channels_10 = ['Fp1','P4', 'P8', 'CP5', 'FC5', 'Fz', 'FC6', 'P7']
channels_11 = ['T7', 'O1', 'CP1', 'F4', 'FC6', 'CP2', 'P8', 'PO10']
channels_12 = ['FT9', 'Fp2', 'C3', 'F4', 'Pz', 'CP6', 'P7', 'F8']
channels_13 = ['PO9', 'Oz', 'T8', 'Fz', 'FT10', 'CP1', 'F4', 'CP6']
channels_14 = ['CP6', 'O2', 'P3', 'Cz', 'FC6', 'T7', 'FC5', 'Fp1']
#channels_15 = ['F3', 'C3', 'Cz', 'P3', 'CP2', 'CP6', 'F8', 'FC6']

init_pop = np.array([channels_0, channels_1, channels_2, channels_3, channels_4, channels_5, 
                    channels_6, channels_7, channels_8, channels_9, channels_10, channels_11, 
                    channels_12, channels_13, channels_14])



In [6]:
def make_init_pop(all_genes, num_genes, num_people):
    # Initialize empty population
    init_pop = np.empty([num_people,num_genes], dtype='<U5')
    person_index = 0

    while person_index!=num_people:
        # Initialize new person
        person = np.empty(num_genes, dtype='<U5')
        gene_index = 0

        while gene_index!=num_genes:
            # Gives a random index
            index = random.randint(0,len(all_genes)-1)
            # Checks if the gene is not already in the gene pool of the person
            if all_genes[index] not in person:
                person[gene_index] = all_genes[index]
                gene_index += 1
        init_pop[person_index] = person
        person_index +=1
    return init_pop

In [7]:
def get_subset(data, all_genes, subset_genes):
    subset_data = np.empty((120, 8, 3200))
    n_genes = 8

    j = 0
    for i in range(len(all_genes)):
        if j < (n_genes + 1) and all_genes[i] in subset_genes:
            subset_data[:,j,:] = data[:,i,:]
            j+=1
    return subset_data

In [8]:
def check_nan(array):
    x = np.isnan(array)
    if True in x:
        print('NAN in array')
        return 0
    print('No NAN found')

In [9]:
def create_labels(dataset):
    labels = load_labels()
    label = pd.concat([labels['t1_math'], labels['t2_math'],
                    labels['t3_math']]).to_numpy()
    label = label.repeat(dataset.shape[1])
    return label

In [10]:
def convert_pop_to_fitness(current_pop,label,n_genes):
    data = np.empty((len(current_pop), 3000, 16))
    new_pop_fitness = np.empty((len(current_pop),3))

    for i in range(len(current_pop)):
        subset_data = get_subset(dataset_ica_2_, channels, current_pop[i])
        dataset = convert_to_epochs(subset_data, n_genes, v.SFREQ)
        features = hjorth_features(dataset, n_genes,v.SFREQ)
        data[i] = features
        check_nan(data[i])

        new_pop_fitness[i] = SVM(data[i], label)

    return new_pop_fitness

In [15]:
def convert_parents_to_fitness(current_pop,label, num_parents_mating, n_genes):
    data = np.empty((num_parents_mating, 3000, 24))
    new_pop_fitness = np.empty((num_parents_mating,3))

    for i in range(num_parents_mating):
        subset_data = get_subset(dataset_ica_2_, channels, current_pop[i])
        dataset = convert_to_epochs(subset_data, n_genes, v.SFREQ)
        features = time_series_features(dataset, n_genes)
        print(features.shape)
        data[i] = features
        check_nan(data[i])

        new_pop_fitness[i] = SVM(data[i], label)

    return new_pop_fitness

In [12]:
num_generations = 10
num_genes_in_person = 8
num_parents_mating = 5
num_people_in_pop = 15


equation_inputs = [4, 1, 1]  # weight for accuracy, sensitivity and specificity 

init_pop = make_init_pop(channels,num_genes_in_person, num_people_in_pop)
print(init_pop)

pop_size = init_pop.shape[0]
curr_pop = init_pop
print(pop_size)



# Creating labels
subset_data = get_subset(dataset_ica_2_, channels, init_pop[0],)
dataset = convert_to_epochs(subset_data, num_genes_in_person, v.SFREQ)
label = create_labels(dataset)

for generation in range(num_generations):
     print(f'Generation number: {generation}')
     # Measuring the fitness of each chromosome in the population.
     curr_pop_fitness = convert_pop_to_fitness(curr_pop,label, num_genes_in_person)
     fitness = ga.cal_pop_fitness(equation_inputs, curr_pop_fitness)
     print(fitness)
     
    # Selecting the best parents in the population for mating.
     parents = ga.select_mating_pool(curr_pop, fitness, num_parents_mating)
 
     # Generating next generation using crossover.
     offspring_crossover = ga.crossover(parents, offspring_size=(pop_size-num_parents_mating, num_genes_in_person))

     # Creating the new population based on the parents and offspring.
     curr_pop[0:num_parents_mating, :] = parents
     curr_pop[num_parents_mating:, :] = offspring_crossover

[['FT10' 'Fp2' 'FC5' 'C4' 'PO10' 'CP1' 'PO9' 'Cz']
 ['P7' 'P8' 'CP1' 'FT9' 'Oz' 'O2' 'F4' 'C3']
 ['PO9' 'Fp1' 'FC5' 'T7' 'C3' 'CP2' 'F7' 'P7']
 ['F7' 'T8' 'FC6' 'C4' 'CP6' 'O2' 'Cz' 'P8']
 ['P7' 'C4' 'FT9' 'P8' 'Pz' 'CP5' 'CP2' 'P3']
 ['P3' 'P7' 'P8' 'F7' 'Fz' 'FC5' 'CP6' 'C4']
 ['Pz' 'P4' 'Fz' 'FT9' 'Fp1' 'P8' 'O2' 'Fp2']
 ['Fp1' 'FC6' 'O2' 'O1' 'FC2' 'C3' 'CP1' 'FT10']
 ['T7' 'Fp1' 'O2' 'FC6' 'CP1' 'FT10' 'O1' 'P3']
 ['F8' 'FT9' 'CP6' 'O2' 'CP5' 'CP2' 'F4' 'FC6']
 ['C3' 'O1' 'P8' 'Cz' 'FT9' 'PO9' 'F8' 'Fz']
 ['Cz' 'O1' 'P8' 'PO10' 'F4' 'P3' 'CP5' 'C4']
 ['PO9' 'Fz' 'FC2' 'P7' 'FC6' 'CP6' 'C3' 'Cz']
 ['P4' 'FT9' 'CP6' 'F8' 'C3' 'CP2' 'FC1' 'Cz']
 ['C3' 'FC5' 'Fp2' 'T7' 'F3' 'P8' 'PO10' 'CP1']]
15
No NAN found

 Confusion matrix:
[[241  70]
 [ 65 224]]
No NAN found

 Confusion matrix:
[[253  58]
 [ 72 217]]
No NAN found

 Confusion matrix:
[[253  58]
 [ 73 216]]
No NAN found

 Confusion matrix:
[[257  54]
 [ 71 218]]
No NAN found

 Confusion matrix:
[[241  70]
 [ 64 225]]
No NAN found


In [13]:
print(parents)
print(offspring_crossover)


[['C3' 'Cz' 'F4' 'T8' 'CP5' 'FT9' 'F7' 'Oz']
 ['FT9' 'Cz' 'T8' 'C3' 'CP5' 'F7' 'F4' 'CP2']
 ['CP2' 'F4' 'F7' 'T8' 'C3' 'FT9' 'CP5' 'Cz']
 ['CP5' 'F7' 'FT9' 'C3' 'Cz' 'F4' 'O2' 'T8']
 ['F7' 'C3' 'Oz' 'CP2' 'F4' 'CP5' 'FT9' 'FC6']]
[['C3' 'Oz' 'F4' 'CP5' 'FT9' 'F7' 'CP2' 'T8']
 ['FT9' 'Oz' 'C3' 'F7' 'CP2' 'T8' 'F4' 'Cz']
 ['CP5' 'Oz' 'F7' 'Cz' 'F4' 'O2' 'C3' 'T8']
 ['F4' 'C3' 'Cz' 'Oz' 'FC6' 'CP5' 'CP2' 'FT9']
 ['CP2' 'F4' 'FT9' 'F7' 'T8' 'C3' 'Cz' 'CP5']
 ['FT9' 'CP2' 'T8' 'C3' 'Cz' 'F7' 'O2' 'F4']
 ['F4' 'Cz' 'T8' 'CP5' 'CP2' 'C3' 'F7' 'FC6']
 ['T8' 'C3' 'Cz' 'F7' 'F4' 'O2' 'CP5' 'FT9']
 ['T8' 'F4' 'C3' 'FT9' 'F7' 'FC6' 'CP2' 'Oz']
 ['F7' 'O2' 'CP5' 'F4' 'CP2' 'FC6' 'Oz' 'FT9']]


In [16]:
fitness = convert_parents_to_fitness(parents, label, num_parents_mating, num_genes_in_person)

print(fitness)

(3000, 24)
No NAN found

 Confusion matrix:
[[235  76]
 [100 189]]
(3000, 24)
No NAN found

 Confusion matrix:
[[249  62]
 [104 185]]
(3000, 24)
No NAN found

 Confusion matrix:
[[249  62]
 [104 185]]
(3000, 24)
No NAN found

 Confusion matrix:
[[234  77]
 [ 97 192]]
(3000, 24)
No NAN found

 Confusion matrix:
[[236  75]
 [101 188]]
[[0.70666667 0.70149254 0.71320755]
 [0.72333333 0.70538244 0.74898785]
 [0.72333333 0.70538244 0.74898785]
 [0.71       0.70694864 0.71375465]
 [0.70666667 0.70029674 0.7148289 ]]
